<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/6%20-%20Generatori%20e%20Images%20Augmentation/generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generatori di Immagini
Spesso capita di dover addestrare una rete neurale su una mole di dati talmente grande da non riuscire a risiedere sulla memoria del nostro PC. A questo scopo tf.keras ci mette a disposizione i generatori, uno strumento che permette di caricare le immagini in batch durante la fase di addestramento.<br>
In questo notebook vedremo come utilizzare i generatori per addestrare una rete neurale in grado di distinguere 5 diversi tipi di dessert, utilizzando un dataset di 10.000 immagini.

## Scarichiamo il Dataset
Il dataset che utilizzeremo è un'estratto del Food101, una raccolta di 101.000 immagini appartenenti a 101 categorie di cibi. Nel nostr caso le categorie sono solamente le seguenti: 
 - cannoli
 - gelato
 - panna cotta
 - tiramisu
 - torta di mele

All'interno dell'archivio troviamo sia la cartella con 10.000 immagini per l'addestramento, 2000 per classe, e la cartella con 1250 immagini per il test, 250 per classe.

In [1]:
!wget https://profession.ai/datasets/dessert.zip
!unzip -qq dessert.zip

--2020-06-29 11:01:23--  https://profession.ai/datasets/dessert.zip
Resolving profession.ai (profession.ai)... 13.225.25.129, 13.225.25.127, 13.225.25.27, ...
Connecting to profession.ai (profession.ai)|13.225.25.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261265207 (249M) [application/zip]
Saving to: ‘dessert.zip’

dessert.zip         100%[===================>] 249.16M  95.4MB/s    in 2.6s    

2020-06-29 11:01:26 (95.4 MB/s) - ‘dessert.zip’ saved [261265207/261265207]



## Importiamo i Moduli

In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Definiamo le Costanti

In [5]:
DATASET_DIR = "dessert/"
BATCH_SIZE = 64

TOT_IMAGES = 10000 
IMG_SIZE = (228, 228)

## Creiamo il Generatore

Possiamo creare il generatore utilizzando la classe *ImageDataGenerator* di tf.keras, al suo interno possiamo definire la dimensione del set di validazione.

In [ ]:
datagen_train = ImageDataGenerator(validation_split=0.1, rescale=1./255,)

tramite il parametro *rescale* abbiamo definito come normalizzare le immagini, cioè dividendo per 255.<br>
Ora per creare i generatori di immagini, utilizziamo il metodo *flow_from_directory*, specificando:
 - il path al set di immagini
 - *target_size*: la dimensione da utilizzare per le immagini
 - *batch_size*: la dimensione di ogni batch di immagini
 - *class_mode*: la tipologia di classificazione, multiclasse (categorical) o binaria (binary).
 - *subset*: qui possiamo specificare se si tratta del set di training o di validazione.

In [ ]:
train_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train", 
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

valid_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

print(train_generator.class_indices)

Found 9000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
Found 1250 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}


Ora creiamo anche un generatore per il Test Set

In [ ]:
datagen_test = ImageDataGenerator(rescale=1./255,)

test_generator = datagen_test.flow_from_directory(
        DATASET_DIR+"test",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

print(test_generator.class_indices)

## Addestriamo la Rete Neurale Convoluzionale sul Generatore
Definiamo l'architettura della rete, trattandosi di un problema abbastanza complesso utilizzeremo diversi strati, insieme a dropout e regolarizzazione L2 per cercare di limitare l'overfiting.

In [24]:
#regularizer = tf.keras.regularizers.L1L2(l1=0.01, l2=0.1)
regularizer = tf.keras.regularizers.L1L2(l2=0.1)

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizer))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 228, 228, 64)      1792      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 228, 228, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 76, 76, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 76, 76, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 76, 76, 32)        18464     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 76, 76, 32)        9248      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 25, 25, 32)       

Per avviare l'addestramento ci basterà passare al metodo *fit* il generatore di addestramento e l'eventuale generatore di validazione. Dobbiamo solo stare attenti a specificare il numero di steps per ogni epoca del Gradient Descent, perché quando utilizziamo i generatori Tensorflow non è in grado di ottenere questo valore automaticamente, solitamente basta utilizzare: NUMERO_DI_IMMAGINI_NEL_SET/DIMENSIONE_DEL_BATCH.

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*0.9 // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*0.1 // BATCH_SIZE),
        epochs=100)

Epoch 1/100
140/140 [==============================] - 83s 596ms/step - loss: 2.6104 - accuracy: 0.1970 - val_loss: 1.6121 - val_accuracy: 0.1969
Epoch 2/100
 47/140 [=========>....................] - ETA: 51s - loss: 1.6107 - accuracy: 0.1955

# Testiamo la Rete sul Generatore
Per testare sul generatore la Rete Neurale che abbiamo addestrato, ci basta passare il generatore al metodo *evaluate.

In [ ]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

 90/141 [==================>...........] - ETA: 12s - loss: 1.6128 - accuracy: 0.1924

KeyboardInterrupt: ignored